In [ ]:
from pandas import DataFrame
import pandas as pd
import geopandas as gpd
import os

# from matplotlib import pyplot as plt
# from matplotlib import font_manager as fm
# import seaborn as sbㅊ
# import numpy as np

In [2]:
# Data load
path = "../Dataset/"  # 파일 경로지정

road1 = pd.read_csv(path + "09._평균속도.csv")
road2 = pd.read_csv(path + "10._추정교통량.csv")
road3 = pd.read_csv(path + "11._혼잡빈도강도.csv")
road4 = pd.read_csv(path + "12._혼잡시간강도.csv")
road5 = gpd.read_file(path + "08.상세도로망_네트워크.geojson")
road13 = gpd.read_file(path + "13._교통사고이력.geojson")

In [3]:
# 정보를 담을 리스트 생성
info_list = []

# 각 데이터프레임의 명칭을 알기 위해 리스트를 딕셔너리 형태로 구성하는 것이 좋습니다.
df_dict = {
    "평균속도": road1,
    "추정교통량": road2,
    "혼잡빈도강도": road3,
    "혼잡시간강도": road4,
}

for name, df in df_dict.items():
    info_list.append(
        {
            "DataSet": name,
            "행 개수": df.shape[0],
            "열 개수": df.shape[1],
            "결측치 합계": df.isna().sum().sum(),  # 전체 결측치 수
            "중복 데이터": df.duplicated().sum(),
        }
    )

# 리스트를 데이터프레임으로 변환
summary_df = pd.DataFrame(info_list)

# 결과 출력
display(summary_df)

,DataSet,행 개수,열 개수,결측치 합계,중복 데이터
0,평균속도,926518,12,0,0
1,추정교통량,926518,14,0,0
2,혼잡빈도강도,39451,10,0,0
3,혼잡시간강도,39451,10,0,0


#### 평균속도/추정교통량

|변수|설명|
|---|---|
v_link_id|상세도로망Level6네트워크의 LinkID(up_v_link or dw_v_link와 매칭)|
road_rank|도로의 등급|
k_length|도로 구간길이(km)|
sido_id|시도코드|
sigungu_id|시군구코드|
emd_id|읍면동코드|
sido_name|시도명|
sigungu_name|시군구명|
emd_name|읍면동명|
velocity_AVRG|모든 차량의 평균속도 (km/h)
probe|차량의 총 통행량(대)
ALL_AADT|모든 차량의 추정교통량 (대/일)
PSCR_AADT|승용차의 추정교통량 (대/일)
BUS_AADT|버스의 추정교통량 (대/일)
FGCR_AADT|화물차의 추정교통량 (대/일)
FRIN_CG|모든 차량의 혼잡빈도강도(%)
TI_CG|모든 차량의 혼잡시간강도(%)

#### 혼잡빈도강도/혼잡시간강도

|변수|설명|
|---|---|
v_link_id|상세도로망Level6네트워크의 LinkID(up_v_link or dw_v_link와 매칭)
road_rank|도로의 등급
k_length|도로 구간길이(km)
sido_id|시도코드
sigungu_id|시군구코드
emd_id|읍면동코드
sido_name|시도명
sigungu_name|시군구명
emd_name|읍면동명
FRIN_CG|모든 차량의 혼잡빈도강도(%)
TI_CG|모든 차량의 혼잡시간강도(%)

In [4]:
# Road1, Road2에서 공통된 모든 컬럼을 리스트로 지정
common_cols1 = [
    "v_link_id",
    "road_rank",
    "k_length",
    "sido_id",
    "sigungu_id",
    "emd_id",
    "sido_name",
    "sigungu_name",
    "emd_name",
    "timeslot",
]

# 공통 컬럼들을 기준으로 병합
merged_df1 = pd.merge(road1, road2, on=common_cols1, how="left")

# Road3, Road4에서 공통된 모든 컬럼을 리스트로 지정
common_cols2 = [
    "v_link_id",
    "road_rank",
    "k_length",
    "sido_id",
    "sigungu_id",
    "emd_id",
    "sido_name",
    "sigungu_name",
    "emd_name",
]

# 공통 컬럼들을 기준으로 병합a
merged_df2 = pd.merge(road3, road4, on=common_cols2, how="left")

all_merge_df = pd.merge(merged_df1, merged_df2, on=common_cols2, how="left")
amdf = all_merge_df.drop(
    [
        "sido_name",
        "sigungu_name",
        "emd_name",
    ],
    axis=1,
)

for df in amdf.columns:
    print(df)
    display(amdf[amdf[df].isnull()])

v_link_id


,v_link_id,road_rank,k_length,sido_id,sigungu_id,emd_id,timeslot,velocity_AVRG,probe,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,FRIN_CG,TI_CG


road_rank


,v_link_id,road_rank,k_length,sido_id,sigungu_id,emd_id,timeslot,velocity_AVRG,probe,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,FRIN_CG,TI_CG


k_length


...

,v_link_id,road_rank,k_length,sido_id,sigungu_id,emd_id,timeslot,velocity_AVRG,probe,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,FRIN_CG,TI_CG


FRIN_CG


,v_link_id,road_rank,k_length,sido_id,sigungu_id,emd_id,timeslot,velocity_AVRG,probe,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,FRIN_CG,TI_CG
113680,57164785401,108,0.277,11000,11240,11240690,0,48.730,904,120,82,10,29,NaN,NaN
113681,57166022401,108,0.325,11000,11240,11240690,0,39.920,2341,368,283,3,81,NaN,NaN
113682,57166908501,108,0.387,11000,11240,11240690,0,47.350,6741,1587,1420,15,152,NaN,NaN
113683,57166864901,108,0.525,11000,11240,11240690,0,53.800,2383,595,532,6,58,NaN,NaN
113684,57171575501,108,0.035,11000,11240,11240790,0,32.800,3505,701,655,5,41,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911803,202400167401,108,0.048,31000,31180,31180660,13,21.330,3,287,262,4,20,NaN,NaN
911804,202400183401,108,0.322,31000,31180,31180660,13,23.000,2,306,281,5,20,NaN,NaN
911805,202400167701,108,0.340,31000,31180,31180660,13,24.500,4,221,192,6,23,NaN,NaN
911806,202400168501,108,0.371,31000,31180,31180660,13,60.920,13,245,212,3,29,NaN,NaN


TI_CG


,v_link_id,road_rank,k_length,sido_id,sigungu_id,emd_id,timeslot,velocity_AVRG,probe,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,FRIN_CG,TI_CG
113680,57164785401,108,0.277,11000,11240,11240690,0,48.730,904,120,82,10,29,NaN,NaN
113681,57166022401,108,0.325,11000,11240,11240690,0,39.920,2341,368,283,3,81,NaN,NaN
113682,57166908501,108,0.387,11000,11240,11240690,0,47.350,6741,1587,1420,15,152,NaN,NaN
113683,57166864901,108,0.525,11000,11240,11240690,0,53.800,2383,595,532,6,58,NaN,NaN
113684,57171575501,108,0.035,11000,11240,11240790,0,32.800,3505,701,655,5,41,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911803,202400167401,108,0.048,31000,31180,31180660,13,21.330,3,287,262,4,20,NaN,NaN
911804,202400183401,108,0.322,31000,31180,31180660,13,23.000,2,306,281,5,20,NaN,NaN
911805,202400167701,108,0.340,31000,31180,31180660,13,24.500,4,221,192,6,23,NaN,NaN
911806,202400168501,108,0.371,31000,31180,31180660,13,60.920,13,245,212,3,29,NaN,NaN


In [ ]:
# 1. 공통 컬럼 정의 (유지보수를 위해 중복 제거)
base_cols = ["v_link_id", "road_rank", "k_length", "sido_id", "sigungu_id", "emd_id"]
name_cols = ["sido_name", "sigungu_name", "emd_name"]
common_cols2 = base_cols + name_cols
common_cols1 = common_cols2 + ["timeslot"]

# 2. 순차적 병합 (Chain Merge)
# road1, 2를 먼저 합치고 그 결과를 3, 4와 차례로 합칩니다.
merged_df = (
    pd.merge(road1, road2, on=common_cols1, how="outer")
    .merge(road3, on=common_cols2, how="outer")
    .merge(road4, on=common_cols2, how="outer")
)

# 3. 불필요한 컬럼 삭제
all_merge_df = merged_df.drop(columns=name_cols)

# 4. 결측치 확인 (가독성 개선)
# 모든 컬럼에 대해 결측치가 있는 행만 요약해서 출력합니다.
for col in all_merge_df.columns:
    null_data = all_merge_df[all_merge_df[col].isnull()]
    if not null_data.empty:
        print(f"--- Column: {col} (Null Count: {len(null_data)}) ---")
        display(null_data.head())  # 너무 많을 수 있으니 상위 5개만 출력

--- Column: FRIN_CG (Null Count: 13576) ---


,v_link_id,road_rank,k_length,sido_id,sigungu_id,emd_id,timeslot,velocity_AVRG,probe,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,FRIN_CG,TI_CG
329007,57164785401,108,0.277,11000,11240,11240690,0,48.730,904,120,82,10,29,NaN,NaN
329008,57164785401,108,0.277,11000,11240,11240690,1,52.030,522,132,89,5,37,NaN,NaN
329009,57164785401,108,0.277,11000,11240,11240690,2,51.620,315,57,40,0,16,NaN,NaN
329010,57164785401,108,0.277,11000,11240,11240690,3,51.820,352,72,49,1,22,NaN,NaN
329011,57164785401,108,0.277,11000,11240,11240690,4,50.260,425,97,78,1,18,NaN,NaN


--- Column: TI_CG (Null Count: 13576) ---


,v_link_id,road_rank,k_length,sido_id,sigungu_id,emd_id,timeslot,velocity_AVRG,probe,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,FRIN_CG,TI_CG
329007,57164785401,108,0.277,11000,11240,11240690,0,48.730,904,120,82,10,29,NaN,NaN
329008,57164785401,108,0.277,11000,11240,11240690,1,52.030,522,132,89,5,37,NaN,NaN
329009,57164785401,108,0.277,11000,11240,11240690,2,51.620,315,57,40,0,16,NaN,NaN
329010,57164785401,108,0.277,11000,11240,11240690,3,51.820,352,72,49,1,22,NaN,NaN
329011,57164785401,108,0.277,11000,11240,11240690,4,50.260,425,97,78,1,18,NaN,NaN


In [11]:
all_merge_df.to_csv("../road.csv", index=False, encoding="utf-8")  # 168Mb > 73Mb
(os.path.getsize("../road.csv") / (1024 * 1024))

71.52339458465576